### Ingestion del archivo "movie_cast.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo JSON utilizando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
movie_cast_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("genderId", IntegerType(), True),
    StructField("castOrder", IntegerType(), True)])


In [0]:
movie_cast_df = spark.read \
    .schema(movie_cast_schema) \
    .option("multiline", "true") \
    .json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json")

#### Paso 3 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
movie_cast_renamed_df = movie_cast_df \
    .withColumnRenamed("movieId","movie_id") \
    .withColumnRenamed("genreId","genre_id") \
    .withColumnRenamed("characterName","character_name") \
    .withColumnRenamed("genderId","gender_id") \
    .withColumnRenamed("castOrder","cast_order") 

In [0]:
movie_cast_dropped_df = movie_cast_renamed_df \
                        .drop("genderId") \
                        .drop("castOrder")

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
movie_cast_final_df = add_ingestion_date(movie_cast_dropped_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment")))\
.withColumn("file_date", lit(v_file_date))

#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
display(movie_cast_final_df)

In [0]:
movie_cast_final_df = movie_cast_final_df.dropDuplicates(["movie_id", "personId"])


In [0]:
#overwrite_partition(movie_cast_final_df,"movie_silver","movie_cast","file_date")
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.personId = src.personId AND tgt.file_date = src.file_date'
merge_delta_lake(movie_cast_final_df,'movie_silver','movie_cast',silver_folder_path,merge_condition,'file_date')

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movie_cast
GROUP BY file_date;